# **Practice #n + 6**  
##### 08.11.24


### **Тема**: Патерны. Комбинация



```mermaid
classDiagram
    class Book {
        +title: str
        +author: Author
        +genre: Genre
        +publicationYear: int
        +getDetails(): str
    }

    class Author {
        +name: str
        +birthYear: int
        +getDetails(): str
    }

    class Genre {
        +name: str
        +description: str
        +getDetails(): str
    }

    class Library{
        +books: list[Book]
        +addBook(book: Book)
        +removeBook(book: Book): void
        +findBook(title: str): Book
    }

    Library --> Book

    class BookAdapter {
        +bookData: dict
        +getDetails(): str
    }

    class BookDecorator {
        +book: Book
        +reviews: list[str]
        +ratings: list[int]
        +addReview(review: str)
        +addRating(rating: int)
        +getDetails
    }

    class LibraryComposite {
        +libraryItems: list[LibraryItem]
        +addItem(item: LibraryItem)
        +removeItem(item: LibraryItem)
        +getDetails(): str
    }

    class LibraryItem {
        +getDetails(): str
    }

    LibraryComposite --> LibraryItem
    LibraryComposite <|-- Library
    BookAdapter ..|> Book
    BookDecorator --|> Book
    LibraryItem <|-- Book
    LibraryItem <|-- Genre
    LibraryItem <|-- Author
```

In [26]:
from  __future__ import annotations
from abc import ABC, abstractmethod



class LibraryItem(ABC):
    @abstractmethod
    def getDetails():
        pass

class Genre(LibraryItem):
    def __init__(self, name: str, description: str):
        self.name = name
        self.description = description

    def getDetails(self) -> str:
        return f"Genre: {self.name} - {self.description}"

class Author(LibraryItem):
    def __init__(self, name: str, birthYear: int):
        self.name = name
        self.birthYear = birthYear

    def getDetails(self) -> str:
        return f"{self.name} (born {self.birthYear})"

class Book(LibraryItem):
    def __init__(self, title: str, author: Author, genre: Genre, publicationYear: int):
        self.title = title
        self.author = author
        self.genre = genre
        self.publicationYear = publicationYear

    def getDetails(self):
        return (f"{self.title} by {self.author.getDetails()}, "
                f"{self.publicationYear} - {self.genre.getDetails()}")




In [27]:
class BookAdapter(Book):
    def __init__(self, bookData: dict):
        author = Author(**bookData["author"])
        genre = Genre(**bookData["genre"])
        super().__init__(bookData["title"], author, genre, \
            bookData["publicationYear"])


In [39]:
class BookDecorator(Book):
    def __init__(self, book: Book):
        self._book = book
        self._reviews = []
        self._rating = []

    def addReview(self, review: str):
        self._reviews.append(review)

    def addRating(self, rating: int):
        self._rating.append(rating)

    def getReviews(self) -> list[str]:
        return self._reviews

    def getAverageRatings(self) -> float:
        return sum(self._rating) / len(self._rating)

    def getDetails(self) -> str:
        details = self._book.getDetails()
        return (f"{details}\nrating: {self.getAverageRatings():.2f}/5\nreviews: "
                f"\n\t* {'\n\t* '.join(self.getReviews())}")



In [48]:
class LibraryComposite:
    def __init__(self) -> None:
        self._items = []

    def addItem(self, item: LibraryItem):
        self._items.append(item)

    def removeItem(self, item: LibraryItem):
        self._items.remove(item)

    def getDetails(self) -> list[str]:
        return "\n--------------------------\n".join(item.getDetails() for item in self._items)

# LibraryComposite()._item

class Library(LibraryComposite):
    def __init__(self) -> None:
        super().__init__()

    @property
    def books(self) -> list[Book]:
        return self._items

    def addBook(self, book: Book):
        if not isinstance(book, Book):
            raise TypeError(f"boook must be an instance of Book, but got {type(book)}")

        self.addItem(book)

    def removeBook(self, book: Book):
        if not isinstance(book, Book):
            raise TypeError(f"boook must be an instance of Book, but got {type(book)}")

        self.removeItem(book)

    def findBook(self, title: str) -> Book:
        for book in self.books:
            if book.title == title:
                return book


In [56]:
author = Author("Lewis Carrol", 1832)
# print(author.getDetails())

genre = Genre("Adventure", "A story of adventure")
# print(genre.getDetails())

book = Book("Alice's Adventures in Wonderland", author, genre, 1865)
# print(book.getDetails())

book_dict = {
    "title": "Harry Potter and the Philosopher's Stone",
    "author": {"name": "J.K. Rowling", "birthYear": 1965},
    "genre":{
        "name": "Fantasy",
        "description": "A story of magic and adventure"
    },
    "publicationYear": 1997
}

book_adapter = BookAdapter(book_dict)
# print(book_adapter.getDetails())

book_decorated = BookDecorator(book_adapter)
book_decorated.addRating(5)
book_decorated.addRating(4)
book_decorated.addRating(2)
book_decorated.addReview("Great book!")
book_decorated.addReview("I loved it!")
book_decorated.addReview("I hate it!")
# print(book_decorated.getDetails())

print("\n*************************************")
lib = Library()
lib.addBook(book)
lib.addBook(book_decorated)
lib.addBook(book_adapter)
print(lib.getDetails())



*************************************
Alice's Adventures in Wonderland by Lewis Carrol (born 1832), 1865 - Genre: Adventure - A story of adventure
--------------------------
Harry Potter and the Philosopher's Stone by J.K. Rowling (born 1965), 1997 - Genre: Fantasy - A story of magic and adventure
rating: 3.67/5
reviews: 
	* Great book!
	* I loved it!
	* I hate it!
--------------------------
Harry Potter and the Philosopher's Stone by J.K. Rowling (born 1965), 1997 - Genre: Fantasy - A story of magic and adventure
